In [1]:
from transformers import MBartForConditionalGeneration, MBart50TokenizerFast
import torch
import random
from copy import deepcopy
import gc
import inspect
import os
from datetime import datetime
import json
import stanza
import xml.etree.ElementTree as ET
from nltk.tree import Tree
from dep_func import depTripleFuncLex
import json

c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:441: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(
c:\Users\Administrator\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\utils\generic.py:309: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  _torch_pytree._register_pytree_node(


In [2]:
# nlp_constituency = stanza.Pipeline('zh', processors='tokenize,pos,constituency')
nlp_dependency = stanza.Pipeline('zh')

2024-09-17 10:50:18 INFO: Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


2024-09-17 10:50:18 INFO: Downloaded file to C:\Users\Administrator\stanza_resources\resources.json
2024-09-17 10:50:18 INFO: "zh" is an alias for "zh-hans"
2024-09-17 10:50:22 INFO: Loading these models for language: zh-hans (Simplified_Chinese):
| Processor    | Package          |
-----------------------------------
| tokenize     | gsdsimp          |
| pos          | gsdsimp_charlm   |
| lemma        | gsdsimp_nocharlm |
| constituency | ctb-51_charlm    |
| depparse     | gsdsimp_charlm   |
| sentiment    | ren_charlm       |
| ner          | ontonotes        |

2024-09-17 10:50:22 INFO: Using device: cuda
2024-09-17 10:50:22 INFO: Loading: tokenize
2024-09-17 10:50:23 INFO: Loading: pos
2024-09-17 10:50:23 INFO: Loading: lemma
2024-09-17 10:50:23 INFO: Loading: constituency
2024-09-17 10:50:24 INFO: Loading: depparse
2024-09-17 10:50:24 INFO: Loading: sentiment
2024-09-17 10:50:25 INFO: Loading: ner
2024-09-17 10:50:26 INFO: Done loading processors!


In [3]:
def get_constituency_parsing(sentence):
    doc = nlp_dependency(sentence)
    sentence_constituency = str(doc.sentences[0].constituency)
    
    return sentence_constituency


#1.0 NP→PN
def NP_PN(t,vector):
    if t.label() == 'NP' and len(t) >= 1:
        if t[0].label() == 'PN':
            vector[0] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_PN(subtree,vector)

# 4.0 NP→DP NP
def NP_DP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'DP' and t[1].label() == 'NP':
            vector[1] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_DP_NP(subtree,vector)

# 6.2 DP→DT
def DP_DT(t,vector):
    if t.label() == 'DP' and len(t) >= 1:
        if t[0].label() == 'DT':
            vector[2] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            DP_DT(subtree,vector)

#6.6 IP→NP VP PU
def IP_NP_VP_PU(t,vector):
    if t.label() == 'IP' and len(t) >= 3:
        if t[0].label() == 'NP' and t[1].label() == 'VP' and t[2].label() == 'PU':
            vector[3] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            IP_NP_VP_PU(subtree,vector)

#6.8 PRN→PU NP PU
def PRN_PU_NP_PU(t,vector):
    if t.label() == 'PRN' and len(t) >= 3:
        if t[0].label() == 'PU' and t[1].label() == 'NP' and t[2].label() == 'PU':
            vector[4] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            PRN_PU_NP_PU(subtree,vector)

#6.8 NP→NR
def NP_NR(t,vector):
    if t.label() == 'NP' and len(t) >= 1:
        if t[0].label() == 'NR':
            vector[5] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NR(subtree,vector)

#10.0 CP_ADVP_IP
def CP_ADVP_IP(t,vector):
    if t.label() == 'CP' and len(t) >= 2:
        if t[0].label() == 'ADVP' and t[1].label() == 'IP':
            vector[6] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            CP_ADVP_IP(subtree,vector)

#10.6 NP_DNP_NP
def NP_DNP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'DNP' and t[1].label() == 'NP':
            vector[7] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_DNP_NP(subtree,vector)
            
#16.4 ADVP_CS
def ADVP_CS(t,vector):
    if t.label() == 'ADVP' and len(t) >= 1:
        if t[0].label() == 'CS':
            vector[8] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            ADVP_CS(subtree,vector)

#16.8 DNP_NP_DEG
def DNP_NP_DEG(t,vector):
    if t.label() == 'DNP' and len(t) >= 2:
        if t[0].label() == 'NP' and t[1].label() == 'DEG': 
            vector[9] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            DNP_NP_DEG(subtree,vector)

def NP_QP_DNP_NP(t,vector):
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'QP' and t[1].label() == 'DNP' and t[2].label() == 'NP':
            vector[10] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_QP_DNP_NP(subtree,vector)

def NP_NP_PRN(t,vector):
    if t.label() == 'NP' and len(t) >= 2:
        if t[0].label() == 'NP' and t[1].label() == 'PRN':
            vector[11] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NP_PRN(subtree,vector)

def NP_NR_CC_NR(t,vector):
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'NR' and t[1].label() == 'CC' and t[2].label() == 'NR':
            vector[12] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NR_CC_NR(subtree,vector)

def NP_NP_CC_NP(t,vector):
    global score, count
    if t.label() == 'NP' and len(t) >= 3:
        if t[0].label() == 'NP' and t[1].label() == 'CC' and t[2].label() == 'NP':
            vector[13] = 1
    for subtree in t:
        if isinstance(subtree, Tree):
            NP_NP_CC_NP(subtree,vector)
            
def get_feature_vector(sentence,vector):
    
    sentence_constituency = get_constituency_parsing(sentence)
    #CFGR features
    tree = Tree.fromstring(sentence_constituency)
    find_NP_PN = NP_PN(tree,vector)
    find_NP_DP_NP = NP_DP_NP(tree,vector)
    find_DP_DT = DP_DT(tree,vector)
    find_IP_NP_VP_PU = IP_NP_VP_PU(tree,vector)
    find_PRN_PU_NP_PU = PRN_PU_NP_PU(tree,vector)
    find_NP_NR = NP_NR(tree,vector)
    find_CP_ADVP_IP = CP_ADVP_IP(tree,vector)
    find_NP_DNP_NP = NP_DNP_NP(tree,vector)
    find_ADVP_CS = ADVP_CS(tree,vector)
    find_DNP_NP_DEG = DNP_NP_DEG(tree,vector)
    #NP features
    find_NP_QP_DNP_NP = NP_QP_DNP_NP(tree,vector)
    find_NP_NP_PRN = NP_NP_PRN(tree,vector)
    find_NP_NR_CC_NR = NP_NR_CC_NR(tree,vector)
    find_NP_NP_CC_NP = NP_NP_CC_NP(tree,vector)

    return vector



In [4]:
def get_dependency_parsing(sentences):
    parsed_words = []
    for sentence in sentences:
        sentence = sentence.strip()  # 去除多余的空白字符
        if not sentence:
            continue
        doc_dependency = nlp_dependency(sentence)
        parsed_words.append(doc_dependency)



In [5]:

def get_feature_vector_dep(sentence,vector):
    doc_dependency = nlp_dependency(sentence)
    a = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','我')
    b = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','将')
    c = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','他')
    d = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','该')
    e = depTripleFuncLex(doc_dependency.sentences[0].words,'NR','case','的')
    f = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','他们')
    g = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','她')
    h = depTripleFuncLex(doc_dependency.sentences[0].words,'他','case','的')
    i = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','nmod:assmod','他')
    j = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','punct','。')
    k = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','但是')
    l = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','你')
    m = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','advmod','如果')
    n = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','mark','的')
    o = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','任何')
    p = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','case','因为')
    q = depTripleFuncLex(doc_dependency.sentences[0].words,'NR','cc','和')
    r = depTripleFuncLex(doc_dependency.sentences[0].words,'NN','det','那些')
    s = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','nsubj','它')
    t = depTripleFuncLex(doc_dependency.sentences[0].words,'VV','dobj','它')
    if a:
        vector[14] = 1
    if b:
        vector[15] = 1
    if c:
        vector[16] = 1
    if d:
        vector[17] = 1
    if e:
        vector[18] = 1
    if f:
        vector[19] = 1
    if g:
        vector[20] = 1
    if h:
        vector[21] = 1
    if i:
        vector[22] = 1
    if j:
        vector[23] = 1
    if k:
        vector[24] = 1
    if l:
        vector[25] = 1
    if m:
        vector[26] = 1
    if n:
        vector[27] = 1
    if o:
        vector[28] = 1
    if p:
        vector[29] = 1
    if q:   
        vector[30] = 1
    if r:
        vector[31] = 1
    if s:
        vector[32] = 1
    if t:
        vector[33] = 1
    return vector


In [6]:
vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]

In [7]:
def read_txt_files_in_directory(directory):
    reslut = []
    # 遍历指定文件夹中的所有文件
    for filename in os.listdir(directory):
        # 检查文件是否为 .txt 文件
        if filename.endswith(".txt"):
            file_path = os.path.join(directory, filename)
            print(f"正在读取文件: {file_path}")
            
            # 逐行读取文件内容
            with open(file_path, 'r', encoding='utf-8') as file:
                for line in file:
                    reslut.append(line.strip())  # 使用 strip() 去除每行末尾的换行符
    return reslut

In [8]:
# 确定运行环境，是cuda还是cpu
ENV = ""
if torch.cuda.is_available():
    ENV="cuda"
else:
    ENV="cpu"

In [9]:


# 保存翻译结果
translated_sens = []

In [10]:
# 原始模型和分词器的初始化
origin_model = MBartForConditionalGeneration.from_pretrained("facebook/mbart-large-50-many-to-many-mmt").to(ENV)
tokenizer = MBart50TokenizerFast.from_pretrained("facebook/mbart-large-50-many-to-many-mmt")
tokenizer.src_lang = "en_XX"


In [11]:
# 自定义激活函数模块
class CustomActivation(torch.nn.Module):
    def __init__(self):
        super(CustomActivation, self).__init__()
    
    def forward(self, x):
        print(x.shape)
        return torch.nn.functional.relu(x)  # 这里你可以使用你希望的激活函数

In [12]:
custom_activation_fn = CustomActivation()

In [13]:
decoder_forward_func = origin_model.model.decoder.layers[11].forward
# encoder_forward_func = origin_model.model.encoder.layers[11].forward

random_number = random.uniform(-1, 1)
# print( inspect.signature(encoder_forward_func) )
# print( inspect.signature(decoder_forward_func) )
t = 1
lasttoken_pos = 0
last_token_state = torch.zeros(5, 1, 1024)


In [14]:
# def new_encoder_activate(hidden_states, attention_mask, layer_head_mask, output_attentions = False):
#     modified_hidden_states = hidden_states
#     # print(hidden_states[:,-2,:])
#     modified_hidden_states[:,-2,:]+=torch.randn(1, 1024).to(ENV)
#     # print(modified_hidden_states[:,-2,:])
#     # modified_hidden_states = torch.nn.functional.gelu(modified_hidden_states)
#     output = encoder_forward_func(
#         modified_hidden_states, attention_mask, 
#         layer_head_mask, output_attentions
#     )
#     final_output = output * 2  # 举例，对输出进行缩放
#     return final_output

In [15]:
with open('.\\processed\\cluster0_steer_final2024-09-16-23-23-27.json', 'r', encoding='utf-8') as f:
    steer_vector = json.load(f)
steer_vector = torch.tensor(steer_vector).to(ENV)

In [16]:

def new_decoder_activate(hidden_states, attention_mask = None, encoder_hidden_states = None, encoder_attention_mask = None,
                         layer_head_mask = None, cross_attn_layer_head_mask = None, past_key_value = None, output_attentions = False,
                         use_cache = True):
    global steer_vector
    # if t % 4 == 0:
    #     random_number = random.uniform(-1, 1)
    #     modified_hidden_states = hidden_states
    #     modified_hidden_states += random_number
    #     modified_hidden_states = torch.nn.functional.gelu(modified_hidden_states)
    # else:
    #     modified_hidden_states = hidden_states  # 加随机数 torch.nn.functional.gelu
    # t+=1
    # t%=4
    modified_hidden_states = hidden_states + steer_vector
    # 调用原始 decoder 层的 forward 方法
    output = decoder_forward_func(
        modified_hidden_states, attention_mask, encoder_hidden_states,
        encoder_attention_mask, layer_head_mask, cross_attn_layer_head_mask,
        past_key_value, output_attentions, use_cache
    )
    # final_output = output * 2  # 举例，对输出进行缩放
    return output

In [17]:
# 激活模型
if 'activated_model' in globals():
    del activated_model
    # 清理未使用的显存
    torch.cuda.empty_cache()
    # 强制垃圾回收
    gc.collect()
    


In [18]:
activated_model = deepcopy(origin_model)


In [19]:
# activated_model.model.decoder.layers[11].activation_fn = custom_activation_fn
activated_model.model.decoder.layers[11].forward = new_decoder_activate

In [20]:
# 谷歌参考译文：农作物中这些金属的浓度因土壤成分而异，并且可能因大量使用含金属肥料和持续的工业污染而加剧。
# src_text = "The concentration of these metals in crops varies depending on soil composition, and can be exacerbated by the extensive use of metal-containing fertilizers and persistent industrial pollution."

# with open("GUM_academic_art.txt", 'r', encoding='utf-8') as file:
#     text = file.readlines()

# print(text)



In [21]:
cluster0_encodes = []
with open('.\\processed\\cluster0_better.json', 'r', encoding='utf-8') as file:
    cluster0 = json.load(file)
for item in cluster0:
    tmp_encoded_input = tokenizer(item, return_tensors="pt").to(ENV)
    cluster0_encodes.append(tmp_encoded_input)

In [22]:
origin_last_token_state = torch.zeros(5, 1, 1024).to(ENV)
for index, encoded_input in enumerate(cluster0_encodes):
    generated_tokens = activated_model.generate(
        **encoded_input, 
        forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
    )
    translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    translated_text = translated_text.replace(" ", "")
    translated_sens.append(translated_text)
    print(translated_text)

因此,所花费的时间和视觉探索艺术品的方式可以说明其相关性、有趣性和甚至其审美吸引力。
它在学术领域有着悠久的历史,但其产生与意义的关键方面尚未得到充分理解。
首先对研究变量和种族分类进行了总结统计。
国家必须保护财产权,提供公共服务,确保健全司法制度。
一方面,采取促进包容性政治机构的政策的社会将发展多元化和集中的制度。
实施包容性机构的后果体现在增加活动、生产力和经济增长中。
如果正是这种状态表现出来的掠夺行为怎么办呢?
在不稳定的财产权、信息不对称以及司法系统作为宽松的执法机制的情况下,外国直接投资难以吸引。
本节简要介绍了GújjolaayEegimaa及其发言者及其品种。
Eegimaa主要在Mof-Ávvi,一个位于西南部塞内加尔前行政区卡萨曼斯首都Ziguinchor的10个村庄的前王国。
在诗歌中syntactic和metrical单元经常匹配。
代理人只能理性地选择专家,如果真正专家与骗子区别的标准是明智的或者众所周知的:
例如,《边境》杂志队似乎合法——至少在我看来——尽管作者应该支付出版费用。
例如,污染可能是因为试图操纵设置的专门知识跟踪系统。
考虑具有授权功能的机构,例如大学、律师协会或同行评审机构。
这里有两种不同的说法:1.是普遍的Huh,2.是单词Huh。
在G发言后,E发言时插嘴huh?。
它提醒人们必须完成对特别的跨语言学研究和更普遍的习得工作。
这不太可能,因为他们说即使第一个训练项目也是说得不错。
针对上述问题,本文通过开发一种高效、经济、不含有毒化学物质的分段小型光发射二极管灯,为社会环境、健康安全及节能问题提供解决方案。
它通常根据分布于供暖和冷却、照明、操作设备等用途进行分类。
使用正确的适当类型的灯泡能提高能源使用效率。
本文提出了地理信息系统应用领域特异的变异演算子。
此外,我们测试了两个GIS应用中定义的操作员。
事实上,一些评论家称复制是研究证据的“金标准”和“科学过程的支柱”。
虽然漂浮船和浮游船和巡航船在不同区域积累了大量实地水盐度数据,但仍难以通过实地测量以高时空分辨率监测全球海洋盐度。
因此,SSS遥感需要高精度的检索模型。
基于理论和实证方法开发了许多粗糙表面排放模型。
虽然有许多理论和实证模型已经开发出来,但有些问题仍未解决。
腰椎后位移使腰椎在站立时更垂直方向,并增加腰椎收缩和在走路时减少膝flexi

In [23]:
print(translated_sens)

['因此,所花费的时间和视觉探索艺术品的方式可以说明其相关性、有趣性和甚至其审美吸引力。', '它在学术领域有着悠久的历史,但其产生与意义的关键方面尚未得到充分理解。', '首先对研究变量和种族分类进行了总结统计。', '国家必须保护财产权,提供公共服务,确保健全司法制度。', '一方面,采取促进包容性政治机构的政策的社会将发展多元化和集中的制度。', '实施包容性机构的后果体现在增加活动、生产力和经济增长中。', '如果正是这种状态表现出来的掠夺行为怎么办呢?', '在不稳定的财产权、信息不对称以及司法系统作为宽松的执法机制的情况下,外国直接投资难以吸引。', '本节简要介绍了GújjolaayEegimaa及其发言者及其品种。', 'Eegimaa主要在Mof-Ávvi,一个位于西南部塞内加尔前行政区卡萨曼斯首都Ziguinchor的10个村庄的前王国。', '在诗歌中syntactic和metrical单元经常匹配。', '代理人只能理性地选择专家,如果真正专家与骗子区别的标准是明智的或者众所周知的:', '例如,《边境》杂志队似乎合法——至少在我看来——尽管作者应该支付出版费用。', '例如,污染可能是因为试图操纵设置的专门知识跟踪系统。', '考虑具有授权功能的机构,例如大学、律师协会或同行评审机构。', '这里有两种不同的说法:1.是普遍的Huh,2.是单词Huh。', '在G发言后,E发言时插嘴huh?。', '它提醒人们必须完成对特别的跨语言学研究和更普遍的习得工作。', '这不太可能,因为他们说即使第一个训练项目也是说得不错。', '针对上述问题,本文通过开发一种高效、经济、不含有毒化学物质的分段小型光发射二极管灯,为社会环境、健康安全及节能问题提供解决方案。', '它通常根据分布于供暖和冷却、照明、操作设备等用途进行分类。', '使用正确的适当类型的灯泡能提高能源使用效率。', '本文提出了地理信息系统应用领域特异的变异演算子。', '此外,我们测试了两个GIS应用中定义的操作员。', '事实上,一些评论家称复制是研究证据的“金标准”和“科学过程的支柱”。', '虽然漂浮船和浮游船和巡航船在不同区域积累了大量实地水盐度数据,但仍难以通过实地测量以高时空分辨率监测全球海洋盐度。', '因此,SSS遥感需要高精度的检索模型。', '基于理论和实证方法

In [24]:
print(len(translated_sens))

695


In [26]:
reslut = []
for index, en_sen in enumerate(cluster0):
    reslut.append({"en": en_sen, "zh": translated_sens[index]})


In [27]:
with open('./processed/enwithzh_cluster0.json', 'w', encoding='utf-8') as file:
    json.dump(reslut, file, ensure_ascii=False, indent=4)

In [25]:
with open('.\\processed\\cluster0_better_translation.json', 'w', encoding='utf-8') as file:
    json.dump(translated_sens, file, ensure_ascii=False, indent=4)


In [24]:
origin_last_token_state =  origin_last_token_state/len(cluster0_encodes)

In [25]:
with open('.\\processed\\cluster0_steering_vector.json', 'r', encoding='utf-8') as file:
    tmp = json.load(file)
vector = torch.tensor(tmp).to(ENV)
cluster0_steering_vector = vector - origin_last_token_state

In [26]:
print(cluster0_steering_vector)

tensor([[[ 1.2451, -0.0054, -0.0275,  ...,  0.0794,  0.0299,  0.2951]],

        [[ 0.9342,  0.4571, -0.3101,  ..., -0.1556,  0.5534,  0.3920]],

        [[ 1.2144,  0.7748, -0.3940,  ..., -0.8829,  0.3880,  0.8258]],

        [[ 0.4213,  0.7908,  0.0991,  ..., -0.6956, -0.0869,  0.9768]],

        [[ 0.4211,  0.3773, -0.0632,  ..., -0.4794,  1.1525,  0.2756]]],
       device='cuda:0')


In [27]:
def decoder_activate(hidden_states, attention_mask = None, encoder_hidden_states = None, encoder_attention_mask = None,
                         layer_head_mask = None, cross_attn_layer_head_mask = None, past_key_value = None, output_attentions = False,
                         use_cache = True):
    global cluster0_steering_vector
    # if t % 4 == 0:
    #     random_number = random.uniform(-1, 1)
    #     modified_hidden_states = hidden_states
    #     modified_hidden_states += random_number
    #     modified_hidden_states = torch.nn.functional.gelu(modified_hidden_states)
    # else:
    #     modified_hidden_states = hidden_states  # 加随机数 torch.nn.functional.gelu
    # t+=1
    # t%=4
    modified_hidden_states = torch.nn.functional.gelu(hidden_states + cluster0_steering_vector)
    # last_token_state = modified_hidden_states
    # 调用原始 decoder 层的 forward 方法
    output = decoder_forward_func(
        modified_hidden_states, attention_mask, encoder_hidden_states,
        encoder_attention_mask, layer_head_mask, cross_attn_layer_head_mask,
        past_key_value, output_attentions, use_cache
    )
    # final_output = output * 2  # 举例，对输出进行缩放
    return output

In [ ]:
activated_model.model.decoder.layers[11].forward = decoder_activate

In [28]:
for index, encoded_input in enumerate(cluster0_encodes):
    generated_tokens = activated_model.generate(
        **encoded_input, 
        forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
    )
    # origin_last_token_state+=last_token_state
    translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
    translated_text = translated_text.replace(" ", "")
    print(translated_text)

因此,它所花费的时间和视觉探索艺术作品的方式可以说明其相关性、有趣性,甚至其审美吸引力。
它在学术上有着悠久的历史,但其生产和意义的许多关键方面尚未得到充分理解。
首先对研究变量和种族分类进行了总结统计。
整个20世纪的第二半期,我们都目睹了机构回归经济分析。
国家必须保护财产权,提供公共服务,确保健全的司法制度。
一方面,采取促进包容性政治机构的政策社会将发展多元化和集中的制度。
在不稳定的财产权、信息不对称和司法系统作为宽松的执法机制的情况下,外国直接投资难以吸引。
本文简要介绍了格鲁伊霍拉伊埃吉马,其发言者及其品种。
Eegimaa主要在Mof-Ávvi,一个位于西南部塞内加尔前行政区卡萨曼斯的首都Ziguinchor的10个村庄的前王国。
语法和度量单位在诗歌中经常相符。
例如,《边境》杂志队似乎合法——至少在我看来——尽管作者应该支付出版费。
例如,污染可能来自试图操纵建立的专家跟踪系统。
考虑具有授权功能的机构,例如大学、律师协会或同行评审机构。
尽管衰变可能发生在头几年内,但一些研究表明,它会逐渐平息,不会再发生损失。
较高的技能水平可能与较少的衰减有关,甚至没有观察到的损失。
这里有两种不同的说法:1.是是普遍的,2.是单词的。
为了支持第一个说法,我们证明了干涉与修补启动的具体功能形式和功能的相似性在语言中比偶然的巧合所承认的要大得多。
在G发言后,E发言者发出插嘴huh?。


In [29]:
for item in cluster0:
    print(item['baseline'])

因此,它所花费的时间和视觉探索艺术作品的方式可以说明其相关性、有趣性,甚至其审美吸引力。
它在学术上有着悠久的历史,但其生产和意义的许多关键方面尚未得到充分理解。
首先对研究变量和种族分类进行了总结统计。
整个20世纪的第二半期,我们都目睹了机构回归经济分析。
国家必须保护财产权,提供公共服务,确保健全的司法制度。
一方面,采取促进包容性政治机构的政策社会将发展多元化和集中的制度。
在不稳定的财产权、信息不对称和司法系统作为宽松的执法机制的情况下,外国直接投资难以吸引。
本文简要介绍了格鲁伊霍拉伊埃吉马,其发言者及其品种。
Eegimaa主要在Mof-Ávvi,一个位于西南部塞内加尔前行政区卡萨曼斯的首都Ziguinchor的10个村庄的前王国。
语法和度量单位在诗歌中经常相符。
例如,《边境》杂志队似乎合法 — — 至少在我看来 — — 尽管作者应该支付出版费。
例如,污染可能来自试图操纵建立的专家跟踪系统。
考虑具有授权功能的机构,例如大学、律师协会或同行评审机构。
尽管衰变可能发生在头几年内,但一些研究表明,它会逐渐平息,不会再发生损失。
较高的技能水平可能与较少的衰减有关,甚至没有观察到的损失。
这里有两种不同的说法: 1. 是 是 普遍的, 2. 是 单词的 。
为了支持第一个说法,我们证明了干涉与修补启动的具体功能形式和功能的相似性在语言中比偶然的巧合所承认的要大得多。
在 G 发言后, E 发言者发出插嘴 huh? 。


In [20]:
# 原文编码
# encoded_input = tokenizer(src_text, return_tensors="pt").to(ENV)
encodes = []
for src_text in en_sens:
    # print(src_text)
    encoded_input = tokenizer(src_text, return_tensors="pt").to(ENV)
    encodes.append(encoded_input)

In [28]:
# # 原模型翻译译文
# for index, encoded_input in enumerate(encodes):
#     generated_tokens = origin_model.generate(
#         **encoded_input, 
#         forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
#     )
#     # print(generated_tokens)
#     translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
#     print(translated_text)
#     if index == 2:
#         break

torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([1, 11, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5

In [18]:
def compute1(l):
    sum =0
    for i in l:
        sum+=i
    return sum

In [20]:
with open(".\\processed\\2024-09-01-15-44-33_processed.json", 'r', encoding='utf-8') as file:
    baseline = json.load(file)

In [25]:
translated_sens=[]
for index, encoded_input in enumerate(encodes):
    print(encoded_input['input_ids'])
    i=1
    zh_tmp = []
    last_token_state_tmp =[]
    fvec_list_tmp = []
    while i > 0:
        # print(encoded_input['input_ids'])
        t = 1
        
        generated_tokens = activated_model.generate(
            **encoded_input, 
            forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
        )
        print(generated_tokens)
        translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
        translated_text = translated_text.replace(" ", "")
        print(translated_text)
        fvc_tmp = get_feature_vector(translated_text,vector)
        fvc_tmp = get_feature_vector_dep(translated_text,vector)
        vector = [0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0]
    #     if compute1(fvc_tmp) < compute1(baseline[index]['fvec']):
    #         zh_tmp.append(translated_text)
    #         tmp = last_token_state.tolist()
    #         last_token_state_tmp.append(tmp)
    #         fvec_list_tmp.append(fvc_tmp)
        i-=1
    # translated_sens.append({'en': en_sens[index], 'baseline': baseline[index]['zh'], 'zh': zh_tmp , 'last_token_state': last_token_state_tmp, 'fvec': fvec_list_tmp})
    if index == 2:
        break
    # if index % 10 == 0:
    #     # 当前时间
    #     current_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")
    #     # 保存翻译结果
    #     with open(f'output/activated{current_time}.json', 'w', encoding='utf-8') as file:
    #         json.dump(translated_sens, file, ensure_ascii=False, indent=4, separators=(',', ':'))
    #     translated_sens=[]



tensor([[250004,  11249,     54,   3395,   6713,     99,    136,  16981,   4927,
            705,      2]], device='cuda:0')
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
tensor([[     2, 250025,      6,  15037,   5733, 161461,    264,  27963,  21443,
             32,      2]], device='cuda:0')
人们如何看待和体验艺术?
tensor([[250004, 130078,  80854,    111,  29458,   4927,  18244,      7,     54,
           1836,  32153,     98,    705,      2]], device='cuda:0')
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5, 1, 4096])
torch.Size([5

In [29]:

# 当前时间
current_time = datetime.now().strftime("%Y-%m-%d-%H-%M-%S")

# 保存翻译结果
with open(f'output/activated{current_time}.json', 'w', encoding='utf-8') as file:
    json.dump(translated_sens, file, ensure_ascii=False, indent=4, separators=(',', ':'))

In [8]:
#每4个token加一次随机数


def new_decoder_activate(hidden_states, attention_mask = None, encoder_hidden_states = None, encoder_attention_mask = None,
                         layer_head_mask = None, cross_attn_layer_head_mask = None, past_key_value = None, output_attentions = False,
                         use_cache = True):
    global t
    
    if t%4 == 0:
        modified_hidden_states = hidden_states
        modified_hidden_states += random_number

        modified_hidden_states = torch.nn.functional.gelu(modified_hidden_states)  # 加随机数 torch.nn.functional.gelu
    else:
        modified_hidden_states = hidden_states
    
    # 调用原始 decoder 层的 forward 方法
    output = decoder_forward_func(
        modified_hidden_states, attention_mask, encoder_hidden_states,
        encoder_attention_mask, layer_head_mask, cross_attn_layer_head_mask,
        past_key_value, output_attentions, use_cache
    )
    t += 1
    #final_output = output * 2  # 举例，对输出进行缩放
    # print(modified_hidden_states.shape)
    return output

In [ ]:
i=1
while i > 0:
    for encoded_input in encodes:
        t = 1
        random_number = random.uniform(-1, 1)
        generated_tokens = activated_model.generate(
            **encoded_input, 
            forced_bos_token_id=tokenizer.lang_code_to_id["zh_CN"]
        )
        translated_text = tokenizer.decode(generated_tokens[0], skip_special_tokens=True)
        print(translated_text)
    print('\n')
    i-=1

In [ ]:
# 准备函数来捕获输出
decoder_outputs = []

def hook_fn(module, input, output):
    decoder_outputs.append(output)

# 注册钩子到解码器的第11层
layer_idx = 10  # 因为索引从0开始，11层就是10
hook = activated_model.model.decoder.layers[layer_idx].register_forward_hook(hook_fn)

# 前向传播
output = activated_model.generate(encodes[1].input_ids, decoder_input_ids=encodes[1].input_ids)

# 移除钩子
hook.remove()

# decoder_outputs 中现在保存了第 11 层的输出
print(decoder_outputs[0][0].shape)  # 检查输出的形状

torch.Size([5, 16, 1024])


In [ ]:
# 创建一个张量
x = torch.tensor([0.5, -0.5, 1.0, -1.0])

# 应用 GELU 激活函数
output = torch.nn.functional.gelu(x)
print(output)

tensor([ 0.3457, -0.1543,  0.8413, -0.1587])


In [ ]:


'''
这个函数实现的功能是重新定义了一个模型（activated_model）
就提供一种修改模型的思路吧，也不用像他一样也这样写成一个函数

  ###
    模型的结构好像是可以直接通过print打印出来的，比如下面的 print(model) or print(activation_model)
    函数签名也是可以打印出来查看的，函数签名就是类似函数原型吧（不知道你还记不记得C语言哈哈哈），具体是用inspect库
    import inspect
    比如下面的orig，它就是一个函数，想要看他的函数签名，就可以直接：
    print(inspect.signature(orig))
  ###

'''
def get_activated_model(model, plus_prompt, minus_prompt, plus_coeff, minus_coeff, layer):
  # 将原来的模型深拷贝一下，直接等于的话就是一个引用，修改之后原来的模型也会被修改
  activation_model = deepcopy(model)
  # 获取了模型的transformer部分的第layer层的forward函数
  orig = model.transformer.h[layer].forward

  activation = plus_prompt
  acts = dict()

  # 这里是重新定义了一个forward函数，其实他啥都没改，就是多了一步，把{activation, val}放进了字典acts
  def new_act(hidden_states, layer_past=None, attention_mask=None, head_mask=None, use_cache=False, output_attentions=False):
    # orig是原来的forward函数
    val = orig(hidden_states, layer_past, attention_mask, head_mask, use_cache, output_attentions)
    acts[activation] = val
    return val

  # 把activation_model的transformer部分的第layer层的forward函数换成了new_act
  activation_model.transformer.h[layer].forward = new_act
  activation = plus_prompt

  input_ids = tokenizer.encode(activation, return_tensors="pt").to("cuda")
  activation_model(input_ids)


  activation = minus_prompt
  input_ids = tokenizer.encode(activation, return_tensors="pt").to("cuda")
  activation_model(input_ids)

  # 这里又是定义了一个forward函数，里面用到了原来的forward函数（就是orig），因为我们不知道原来的forward函数的实现细节，所以只能这么直接地去调用，然后在他原来返回值的基础上去修改（你看下面）
  def add_acts(hidden_states, layer_past=None, attention_mask=None, head_mask=None, use_cache=False, output_attentions=False):
    val = orig(hidden_states, layer_past, attention_mask, head_mask, use_cache, output_attentions)

    # 得到了原来forward函数的返回值之后，再进行一些修改（干预）
    a = val[0] + plus_coeff* acts[plus_prompt][0] - minus_coeff*acts[minus_prompt][0]
    #x = val[1][0] +  acts['boy'][1][0] - acts['girl'][1][0]
    #y = val[1][1] + acts['boy'][1][1] -  acts['girl'][1][1]

    # 再拼接回去，return出来
    return (a, val[1])

  # 又是一次替换
  activated_model = deepcopy(model)
  activated_model.transformer.h[layer].forward = add_acts


  return activated_model